In [1]:
import intake

## Walkthrough

In [2]:
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
col

<Intake-esm catalog with 3633 dataset(s) from 258775 asset(s)>

### Dataset keys

When a catalog is opened, `intake-esm` uses the `aggregation` information to construct keys for all possible `xarray.Datasets`

In [3]:
len(col.keys())

3633

In [4]:
# Take a look at the first 10 keys
col.keys()[:10]

['AerChemMIP.AS-RCEC.TaiESM1.histSST.AERmon.gn',
 'AerChemMIP.BCC.BCC-ESM1.histSST.AERmon.gn',
 'AerChemMIP.BCC.BCC-ESM1.histSST.Amon.gn',
 'AerChemMIP.BCC.BCC-ESM1.piClim-CH4.Amon.gn',
 'AerChemMIP.BCC.BCC-ESM1.piClim-NTCF.Amon.gn',
 'AerChemMIP.BCC.BCC-ESM1.piClim-control.AERmon.gn',
 'AerChemMIP.BCC.BCC-ESM1.piClim-control.Amon.gn',
 'AerChemMIP.BCC.BCC-ESM1.ssp370.AERmon.gn',
 'AerChemMIP.BCC.BCC-ESM1.ssp370.Amon.gn',
 'AerChemMIP.BCC.BCC-ESM1.ssp370.LImon.gn']

In [5]:
# Key template
col.key_template

'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

In [6]:
# We can easily find whether a key is in our collection or not
'AerChemMIP.BCC.BCC-ESM1.ssp370.LImon.gn' in col

True

In [7]:
'DO_NOT_EXIST' in col

False

### __getitem__: provides access to a single aggregated dataset

We can retrieve a subset of the catalog by specifying which key we are interested in. This returns an instance of `intake.catalog.local.LocalCatalogEntry`:

In [8]:
col_subset = col['AerChemMIP.BCC.BCC-ESM1.ssp370.AERmon.gn']
print(col_subset)

name: AerChemMIP.BCC.BCC-ESM1.ssp370.AERmon.gn
container: xarray
plugin: ['esm_group']
description: 
direct_access: True
user_parameters: []
metadata: 
args: 
  df:     activity_id institution_id source_id experiment_id member_id table_id  \
114  AerChemMIP            BCC  BCC-ESM1        ssp370  r1i1p1f1   AERmon   
115  AerChemMIP            BCC  BCC-ESM1        ssp370  r1i1p1f1   AERmon   
116  AerChemMIP            BCC  BCC-ESM1        ssp370  r1i1p1f1   AERmon   
117  AerChemMIP            BCC  BCC-ESM1        ssp370  r1i1p1f1   AERmon   
118  AerChemMIP            BCC  BCC-ESM1        ssp370  r1i1p1f1   AERmon   
119  AerChemMIP            BCC  BCC-ESM1        ssp370  r1i1p1f1   AERmon   

    variable_id grid_label                                             zstore  \
114       mmrbc         gn  gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...   
115     mmrdust         gn  gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...   
116       mmroa         gn  gs://cmip6/AerChemMIP/BC

In [9]:
type(col_subset)

intake.catalog.local.LocalCatalogEntry

In [10]:
col_subset.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,_data_format_
114,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,AERmon,mmrbc,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN,zarr
115,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,AERmon,mmrdust,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN,zarr
116,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,AERmon,mmroa,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN,zarr
117,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,AERmon,mmrso4,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN,zarr
118,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,AERmon,mmrss,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN,zarr
119,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,AERmon,od550aer,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/ssp370/r1i1...,NaN,zarr


All these asssets are compatible i.e. they can be aggregated into a single xarray dataset.

### Load an entry into an xarray dataset via `.to_dask()`

Invoking `to_dask()` method on an instance of `intake.catalog.local.LocalCatalogEntry` will load assets into an xarray dataset. Note that this method returns an `xarray.Dataset`

In [11]:
ds = col_subset(zarr_kwargs={'consolidated': True, 'decode_times': False}).to_dask()

In [12]:
ds

<xarray.Dataset>
Dimensions:     (bnds: 2, lat: 64, lev: 26, lon: 128, member_id: 1, time: 492)
Coordinates:
    a_bnds      (lev, bnds) float64 dask.array<chunksize=(26, 2), meta=np.ndarray>
    time_bnds   (time, bnds) float64 dask.array<chunksize=(492, 2), meta=np.ndarray>
    lon_bnds    (lon, bnds) float64 dask.array<chunksize=(128, 2), meta=np.ndarray>
    b_bnds      (lev, bnds) float64 dask.array<chunksize=(26, 2), meta=np.ndarray>
    lat_bnds    (lat, bnds) float64 dask.array<chunksize=(64, 2), meta=np.ndarray>
    lev_bnds    (lev, bnds) float64 dask.array<chunksize=(26, 2), meta=np.ndarray>
  * time        (time) int64 0 708 1416 2148 ... 356220 356952 357684 358416
  * lev         (lev) float64 0.9926 0.9706 0.9296 ... 0.01397 0.007389 0.003545
  * lon         (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat         (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * member_id   (member_id) <U8 'r1i1p1f1'
    wavelength  float64 ...
Dimensions without coordinates: bnds
Data variables:
    b           (lev) float64 dask.array<chunksize=(26,), meta=np.ndarray>
    p0          float64 1e+05
    ps          (time, lat, lon) float32 dask.array<chunksize=(56, 64, 128), meta=np.ndarray>
    a           (lev) float64 dask.array<chunksize=(26,), meta=np.ndarray>
    mmrbc       (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    mmrdust     (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    mmroa       (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    mmrso4      (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    mmrss       (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    od550aer    (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 492, 64, 128), meta=np.ndarray>
Attributes:
    variant_label:           r1i1p1f1
    experiment:              gap-filling scenario reaching 7.0 based on SSP3
    external_variables:      areacella
    tracking_id:             hdl:21.14100/cb10f6fc-092f-4b4f-8f85-42608bfc018...
    comment:                 Transient future coupled-ocean experiments start...
    parent_mip_era:          CMIP6
    realization_index:       1
    source_id:               BCC-ESM1
    run_variant:             forcing:  greenhouse gases,aerosol emission,sola...
    data_specs_version:      01.00.27
    grid_label:              gn
    branch_time_in_parent:   2015.0
    source:                  BCC-ESM 1 (2017):   aerosol: none  atmos: BCC_AG...
    parent_source_id:        BCC-ESM1
    institution_id:          BCC
    sub_experiment_id:       none
    parent_experiment_id:    historical
    realm:                   aerosol
    description:             SSP3-7.0
    branch_method:           Standard
    activity_id:             AerChemMIP
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.BCC.BCC-ESM...
    mip_era:                 CMIP6
    table_info:              Creation Date:(30 July 2018) MD5:e53ff52009d0b97...
    grid:                    T42
    history:                 2019-07-18T02:12:10Z ; CMOR rewrote data to be c...
    table_id:                AERmon
    parent_activity_id:      CMIP
    nominal_resolution:      250 km
    experiment_id:           ssp370
    references:              Model described by Tongwen Wu et al. (JGR 2013; ...
    product:                 model-output
    institution:             Beijing Climate Center, Beijing 100081, China
    physics_index:           1
    title:                   BCC-ESM1 output prepared for CMIP6
    parent_variant_label:    r1i1p1f1
    parent_time_units:       days since 2015-01-01
    initialization_index:    1
    cmor_version:            3.3.2
    contact:                 Dr. Tongwen Wu(twwu@cma.gov.cn)
    license:          

As you can see, multiple variables from multiple zarr stores have been merged into a single dataset.

### Search()

The search method still returns an instance of `intake_esm.core.esm_datastore`

In [13]:
col_x = col.search(activity_id='AerChemMIP', source_id='BCC-ESM1', table_id='AERmon', experiment_id='ssp370')
col_x

<Intake-esm catalog with 1 dataset(s) from 6 asset(s)>

In [14]:
col_x.keys()

['AerChemMIP.BCC.BCC-ESM1.ssp370.AERmon.gn']

### to_dataset_dict()

When working with an instance of `intake_esm.core.esm_datastore`, the appropriate way to load all assets into xarray datasets is via `to_dataset_dict()` method.

In [15]:
dsets = col_x.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False})
dsets.keys()

Dataset(s):   0%|                                        | 0/1 [00:00<?, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


Dataset(s): 100%|████████████████████████████████| 1/1 [00:07<00:00,  7.88s/it]


dict_keys(['AerChemMIP.BCC.BCC-ESM1.ssp370.AERmon.gn'])

In [16]:
# Extract a single key from dictionary of datasets
ds = dsets['AerChemMIP.BCC.BCC-ESM1.ssp370.AERmon.gn']
ds

<xarray.Dataset>
Dimensions:     (bnds: 2, lat: 64, lev: 26, lon: 128, member_id: 1, time: 492)
Coordinates:
    a_bnds      (lev, bnds) float64 dask.array<chunksize=(26, 2), meta=np.ndarray>
    time_bnds   (time, bnds) float64 dask.array<chunksize=(492, 2), meta=np.ndarray>
    lon_bnds    (lon, bnds) float64 dask.array<chunksize=(128, 2), meta=np.ndarray>
    b_bnds      (lev, bnds) float64 dask.array<chunksize=(26, 2), meta=np.ndarray>
    lat_bnds    (lat, bnds) float64 dask.array<chunksize=(64, 2), meta=np.ndarray>
    lev_bnds    (lev, bnds) float64 dask.array<chunksize=(26, 2), meta=np.ndarray>
  * time        (time) int64 0 708 1416 2148 ... 356220 356952 357684 358416
  * lev         (lev) float64 0.9926 0.9706 0.9296 ... 0.01397 0.007389 0.003545
  * lon         (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat         (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * member_id   (member_id) <U8 'r1i1p1f1'
    wavelength  float64 ...
Dimensions without coordinates: bnds
Data variables:
    b           (lev) float64 dask.array<chunksize=(26,), meta=np.ndarray>
    p0          float64 1e+05
    ps          (time, lat, lon) float32 dask.array<chunksize=(56, 64, 128), meta=np.ndarray>
    a           (lev) float64 dask.array<chunksize=(26,), meta=np.ndarray>
    mmrbc       (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    mmrdust     (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    mmroa       (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    mmrso4      (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    mmrss       (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 56, 26, 64, 128), meta=np.ndarray>
    od550aer    (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 492, 64, 128), meta=np.ndarray>
Attributes:
    variant_label:           r1i1p1f1
    experiment:              gap-filling scenario reaching 7.0 based on SSP3
    external_variables:      areacella
    tracking_id:             hdl:21.14100/cb10f6fc-092f-4b4f-8f85-42608bfc018...
    comment:                 Transient future coupled-ocean experiments start...
    parent_mip_era:          CMIP6
    realization_index:       1
    source_id:               BCC-ESM1
    run_variant:             forcing:  greenhouse gases,aerosol emission,sola...
    data_specs_version:      01.00.27
    grid_label:              gn
    branch_time_in_parent:   2015.0
    source:                  BCC-ESM 1 (2017):   aerosol: none  atmos: BCC_AG...
    parent_source_id:        BCC-ESM1
    institution_id:          BCC
    sub_experiment_id:       none
    parent_experiment_id:    historical
    realm:                   aerosol
    description:             SSP3-7.0
    branch_method:           Standard
    activity_id:             AerChemMIP
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.BCC.BCC-ESM...
    mip_era:                 CMIP6
    table_info:              Creation Date:(30 July 2018) MD5:e53ff52009d0b97...
    grid:                    T42
    history:                 2019-07-18T02:12:10Z ; CMOR rewrote data to be c...
    table_id:                AERmon
    parent_activity_id:      CMIP
    nominal_resolution:      250 km
    experiment_id:           ssp370
    references:              Model described by Tongwen Wu et al. (JGR 2013; ...
    product:                 model-output
    institution:             Beijing Climate Center, Beijing 100081, China
    physics_index:           1
    title:                   BCC-ESM1 output prepared for CMIP6
    parent_variant_label:    r1i1p1f1
    parent_time_units:       days since 2015-01-01
    initialization_index:    1
    cmor_version:            3.3.2
    contact:                 Dr. Tongwen Wu(twwu@cma.gov.cn)
    license:          

### Summary

With https://github.com/NCAR/intake-esm/pull/194:

```python
dset_dict = col.to_dataset_dict()
ds = dset_dict['AerChemMIP.BCC.BCC-ESM1.ssp370.AERmon.gn']
```

is equivalent to 

```python
ds = col['AerChemMIP.BCC.BCC-ESM1.ssp370.AERmon.gn'].to_ask()
```


